In [1]:
!pip install arxiv


In [6]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper


In [15]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
wikitool = WikipediaQueryRun(api_wrapper=api_wrapper)
wikitool.name

'wikipedia'

In [8]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [10]:
loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size= 1000, chunk_overlap=200).split_documents(docs)
vectordb = FAISS.from_documents(documents,OllamaEmbeddings(model="gemma:2b") )
retriever = vectordb.as_retriever()

First tool with wikipedia (webbaseloader)

In [12]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,"langsmith_search",
                      "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!")

Second tool with arxiv


In [13]:
## Arxiv Tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper=ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [16]:
#combine all tools

tools=[wikitool,arxiv,retriever_tool]

In [17]:
# agents that choose from each tools

from langchain.agents import create_openai_tools_agent
from dotenv import load_dotenv
from langchain_community.llms import Ollama
import os

load_dotenv()

os.environ["LANGCHAIN_TRACING_V2"]="true"
api_key = os.getenv('LANGCHAIN_API_KEY')

if api_key is None:
    raise ValueError("La clé API LANGCHAIN_API_KEY n'a pas été définie dans les variables d'environnement.")
os.environ["LANGCHAIN_API_KEY"]=api_key


llm=Ollama(model="gemma:2b")


In [18]:
#take a prompt already done

from langchain import hub
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [19]:
### Agents
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)

In [20]:
#Agent executer 
from langchain.agents import AgentExecutor
agent = AgentExecutor(agent=agent , tools=tools,verbose=True)


In [23]:
agent.invoke({"input":"Tell me about Langsmith"})




> Entering new AgentExecutor chain...


ValueError: Ollama call failed with status code 400. Details: {"error":"invalid options: tools"}